Prepare the data

In [1]:
import torch
from datasets import load_dataset

print(torch.cuda.device_count())

d:\Project\move-review-sentiment-analysis-model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1


In [2]:
# imdb_dataset_path = "D:/Project/move-review-sentiment-analysis-model/aclImdb"
cache_dir = r"D:\Project\move-review-sentiment-analysis-model\aclImdb"
ds = load_dataset("stanfordnlp/imdb", cache_dir=cache_dir)

In [3]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [4]:
train_ds = load_dataset("stanfordnlp/imdb", cache_dir=cache_dir, split="train")
test_ds = load_dataset("stanfordnlp/imdb", cache_dir=cache_dir, split="test")

Label:  
0: neg  
1: pos

In [5]:
train_ds.features

{'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}

In [6]:
print(train_ds[0])
print(train_ds[1])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [7]:
df_train = train_ds.to_pandas()
df_test = test_ds.to_pandas()

In [8]:
print("Info: ", df_train.info())
print(df_train["label"].nunique())
print(df_test["label"].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    25000 non-null  object
 1   label   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB
Info:  None
2
2


In [9]:
# Class balance
print("Train dataset class balance: ",
    (df_train["label"].value_counts(normalize=True) * 100).round(2).astype(str) + "%"
)

print("Test dataset class balance: ",
    (df_train["label"].value_counts(normalize=True) * 100).round(2).astype(str) + "%"
)

Train dataset class balance:  label
0    50.0%
1    50.0%
Name: proportion, dtype: object
Test dataset class balance:  label
0    50.0%
1    50.0%
Name: proportion, dtype: object


In [13]:
print(df_train.head(n=10))

# Longest text
print(df_train["text"].apply(len).idxmax())
print(df_train.iloc[13756]["text"])
print("text length: ", len(df_train.iloc[13756]["text"]))

                                                text  label
0  I rented I AM CURIOUS-YELLOW from my video sto...      0
1  "I Am Curious: Yellow" is a risible and preten...      0
2  If only to avoid making this type of film in t...      0
3  This film was probably inspired by Godard's Ma...      0
4  Oh, brother...after hearing about this ridicul...      0
5  I would put this at the top of my list of film...      0
6  Whoever wrote the screenplay for this movie ob...      0
7  When I first saw a glimpse of this movie, I qu...      0
8  Who are these "They"- the actors? the filmmake...      0
9  This is said to be a personal film for Peter B...      0
13756
Match 1: Tag Team Table Match Bubba Ray and Spike Dudley vs Eddie Guerrero and Chris Benoit Bubba Ray and Spike Dudley started things off with a Tag Team Table Match against Eddie Guerrero and Chris Benoit. According to the rules of the match, both opponents have to go through tables in order to get the win. Benoit and Guerrero heat

In [ ]:
# Average text length
lengths = train_ds.map(lambda x: {"length": len(x["text"])})["length"]
len(lengths)

import numpy as np
print("average text lenght: ", np.mean(lengths)) 

# Average text is 1325 character, about 300 token

average text lenght:  1325.06964


In [ ]:
# Testing tokenizer for bert
from transformers import AutoTokenizer

text = df_train["text"][0]
print(text)

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
embedding_output = tokenizer(text)
print(embedding_output)


I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [ ]:
label_map = {
    0: "negative",  
    1: "positive"
}